In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/layer7/valid.csv
/kaggle/input/layer7/train.csv
/kaggle/input/layer7/test.csv


In [2]:
import pandas as pd
import numpy as np
from pandas import Series

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [3]:
train_df = pd.read_csv('/kaggle/input/layer7/train.csv')
valid_df = pd.read_csv('/kaggle/input/layer7/valid.csv')
test_df = pd.read_csv('/kaggle/input/layer7/test.csv')

In [4]:
train_df.shape

(28520, 772)

In [5]:
train_df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
0,0.186257,-0.058807,0.024632,-0.163933,-0.146699,0.035889,0.111708,-0.162861,0.028249,-0.098063,...,0.055629,-0.010358,0.125754,0.011648,0.079197,0.093215,45,NaN,1,6
1,0.063431,-0.023597,0.068057,-0.252915,-0.061094,-0.027316,0.135747,-0.168147,0.091236,-0.078473,...,-0.014893,0.071721,0.018918,0.100032,-0.083042,0.088615,45,NaN,1,6
2,0.034962,0.035816,-0.029753,-0.094607,-0.017576,-0.053074,0.040121,-0.007932,0.097872,-0.024042,...,0.012415,0.015215,0.083808,0.031312,-0.056277,0.064702,45,NaN,1,6
3,0.033772,0.085612,0.067488,-0.073953,-0.180646,-0.024512,0.242879,-0.023374,-0.059999,0.002006,...,-0.078246,-0.032903,0.082949,-0.020659,0.082274,-0.050164,45,NaN,1,6
4,0.134305,0.062096,0.106920,-0.089327,0.117093,-0.077107,0.152579,0.047529,-0.015998,-0.110657,...,-0.094629,0.069718,0.014379,0.048124,0.007586,-0.016980,45,NaN,1,6


In [6]:
missing_columns = train_df.columns[train_df.isnull().any()]
missing_counts = train_df[missing_columns].isnull().sum()

print('Missing Columns and Counts')
for column in missing_columns:
    print( str(column) +' : '+ str(missing_counts[column]))

Missing Columns and Counts
label_2 : 480


In [7]:
train_data = train_df.copy()
valid_data = valid_df.copy()
test_data = test_df.copy()

In [8]:
train_df.describe()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
count,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,...,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28520.000000,28040.000000,28520.000000,28520.000000
mean,0.065874,0.028860,0.017909,-0.063664,-0.152100,-0.047104,0.103360,-0.019966,0.053955,-0.066990,...,-0.027808,-0.036166,0.020930,0.033553,0.021910,0.002223,30.498843,27.975107,0.799299,5.997125
std,0.077031,0.052005,0.066953,0.063553,0.076321,0.060004,0.080915,0.064415,0.055282,0.055525,...,0.062439,0.059810,0.055272,0.061426,0.057703,0.077277,17.328389,5.735913,0.400532,2.375567
min,-0.208012,-0.255053,-0.266606,-0.343267,-0.513899,-0.310102,-0.219100,-0.328737,-0.144846,-0.296431,...,-0.277574,-0.292493,-0.246473,-0.225174,-0.202081,-0.322757,1.000000,22.000000,0.000000,0.000000
25%,0.017124,-0.006248,-0.030998,-0.103098,-0.202183,-0.085325,0.041097,-0.062285,0.016961,-0.102529,...,-0.068011,-0.077156,-0.015498,-0.008668,-0.019020,-0.043847,15.000000,25.000000,1.000000,6.000000
50%,0.067048,0.031829,0.014992,-0.063066,-0.150201,-0.043714,0.102243,-0.019105,0.058116,-0.062655,...,-0.026689,-0.033144,0.024588,0.029152,0.016737,0.012354,30.000000,27.000000,1.000000,6.000000
75%,0.117308,0.063893,0.063480,-0.022328,-0.101837,-0.004591,0.162044,0.022008,0.092691,-0.028027,...,0.011446,0.007627,0.059094,0.073403,0.058569,0.057865,46.000000,30.000000,1.000000,6.000000
max,0.335540,0.235086,0.304683,0.242093,0.167758,0.206523,0.402070,0.270795,0.276515,0.255341,...,0.224582,0.199627,0.254224,0.312076,0.260679,0.279346,60.000000,61.000000,1.000000,13.000000


In [9]:
from sklearn.preprocessing import RobustScaler # eliminate outliers

x_train = {}
x_valid = {}
x_test = {}

y_train = {}
y_valid = {}
y_test = {}

#create dictionaries for each label
for target_label in ['label_1','label_2','label_3','label_4']:

  if target_label == "label_2":
    train = train_df[train_df['label_2'].notna()]
    valid = valid_df[valid_df['label_2'].notna()]
  else:
    train = train_df
    valid = valid_df

  test = test_df

  scaler = RobustScaler()

  x_train[target_label] = pd.DataFrame(scaler.fit_transform(train.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_train[target_label] = train[target_label]

  x_valid[target_label] = pd.DataFrame(scaler.transform(valid.drop(['label_1','label_2','label_3','label_4'], axis=1)), columns=[f'feature_{i}' for i in range(1,769)])
  y_valid  [target_label] = valid[target_label]

  x_test[target_label] = pd.DataFrame(scaler.transform(test.drop(["ID"],axis=1)), columns=[f'feature_{i}' for i in range(1,769)])

In [10]:
x_train_df = x_train['label_4'].copy()
y_train_df = y_train['label_4'].copy()

x_valid_df = x_valid['label_4'].copy()
y_valid_df = y_valid['label_4'].copy()

x_test_df = x_test['label_4'].copy()

# Cross Validation

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold

# Perform cross-validation
scores = cross_val_score(SVC(), x_train_df, y_train_df, cv=5, scoring='accuracy')

mean_accuracy = scores.mean()
std_accuracy = scores.std()
# Print the cross-validation scores
print('Support Vector Machines')
print('\n')
print("Cross-validation scores:", scores)
print(f"Mean Accuracy: {mean_accuracy:.2f}")
print(f"Standard Deviation: {std_accuracy:.2f}")

Support Vector Machines


Cross-validation scores: [0.92198457 0.92321178 0.94126928 0.94249649 0.93267882]
Mean Accuracy: 0.93
Standard Deviation: 0.01


# Feature Engineering

In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.975, svd_solver='full')
pca.fit(x_train_df)
x_train_df_pca = pd.DataFrame(pca.transform(x_train_df))
x_valid_df_pca = pd.DataFrame(pca.transform(x_valid_df))
x_test_df_pca = pd.DataFrame(pca.transform(x_test_df))
print('Shape after PCA: ',x_train_df_pca.shape)

Shape after PCA:  (28520, 443)


In [13]:
from sklearn import metrics

# SVM

In [14]:
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

classifier = svm.SVC(kernel='linear', C=1)

classifier.fit(x_train_df_pca, y_train_df)

y_valid_pred = classifier.predict(x_valid_df_pca)

print("acc_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

acc_score:  0.9293333333333333


In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
import numpy as np

param_dist = {
    'C': [100,10,1,0,0.1,0.01],
    'kernel': ['rbf','linear','poly','sigmoid'],
    'gamma': ['scale','auto'],
    'degree': [1,2,3,4],
    'class_weight' : ['none','balanced']
}

svm = SVC()

random_search = RandomizedSearchCV(
    svm, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1, random_state=42, scoring='accuracy'
)

random_search.fit(x_train_df_pca, y_train_df)

best_params = random_search.best_params_
best_model = random_search.best_estimator_

print("best parameters:", best_params)

best parameters: {'kernel': 'rbf', 'gamma': 'scale', 'degree': 4, 'class_weight': 'balanced', 'C': 100}


Label 1 
best parameters: {'kernel': 'rbf', 'gamma': 'scale', 'degree': 4, 'class_weight': 'balanced', 'C': 100}


In [16]:
from sklearn import svm

classifier = svm.SVC(kernel=best_params['kernel'], C=best_params['C'], gamma=best_params['gamma'], degree=best_params['degree'], class_weight=best_params['class_weight'])

classifier.fit(x_train_df_pca, y_train_df)

y_valid_pred = classifier.predict(x_valid_df_pca)

print("acc_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

y_test_predict_after_pca = classifier.predict(x_test_df_pca)



acc_score:  0.972


# RandomForest

In [17]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=100, random_state=42)

classifier.fit(x_train_df, y_train_df)

y_valid_pred = classifier.predict(x_valid_df)

print("accuracy_score: ",metrics.accuracy_score(y_valid_df, y_valid_pred))

y_test_pred = classifier.predict(x_test_df)

accuracy_score:  0.7653333333333333


# CSV Creation

In [18]:
output_df=pd.DataFrame(columns=["ID","label_1","label_2","label_3","label_4"])

In [19]:
IDs = list(i for i in range(1, len(test_df)+1))
output_df["ID"] = IDs

In [20]:
output_df["label_4"] = y_test_predict_after_pca

In [21]:
output_df

,ID,label_1,label_2,label_3,label_4
0,1,NaN,NaN,NaN,2
1,2,NaN,NaN,NaN,8
2,3,NaN,NaN,NaN,6
3,4,NaN,NaN,NaN,6
4,5,NaN,NaN,NaN,6
...,...,...,...,...,...
739,740,NaN,NaN,NaN,6
740,741,NaN,NaN,NaN,2
741,742,NaN,NaN,NaN,6
742,743,NaN,NaN,NaN,12


In [22]:
output_df.to_csv('/kaggle/working/output7_4.csv',index=False)